In [17]:
import numpy as nmp 
import math
import os
import os.path

In [18]:
def first_div_diff(x1,x0,y1,y0):
    return (y1-y0)/(x1-x0)


def second_div_diff(x2,x1,x0,y2,y1,y0):
    return (first_div_diff(x2,x1,y2,y1)-first_div_diff(x1,x0,y1,y0))/(x2-x0)


def formula_select(arg,x_list):
    if (arg<=((x_list[0]+x_list[len(x_list)-1])/2)):
        return 1
    else:
        return 2

def get_necessary_index(arg,x_list):
    for i in range(0,len(x_list)):
        if (x_list[i] > arg):
            required_index = i
            break
    return required_index


def check_for_newton(x_list):
    h = round((x_list[1]-x_list[0]),10)
    for i in range(1,len(x_list)):
        if (round((x_list[i]-x_list[i-1]),10) != h):
            return False 
    return True

In [20]:
def linear_interpolation(arg,x_list,y_list):
    required_index = get_necessary_index(arg,x_list)
    c_A = (y_list[required_index]-y_list[required_index-1])/(x_list[required_index]-x_list[required_index-1])
    c_B = y_list[required_index-1]-(c_A*x_list[required_index-1])
    return (c_A*arg + c_B)


def polynomial_interpolation(arg,x_list,y_list):
    required_index = get_necessary_index(arg,x_list)
    coefficients = nmp.array([[x_list[required_index-2]*x_list[required_index-2],x_list[required_index-2],1], 
                       [x_list[required_index-1]*x_list[required_index-1],x_list[required_index-1],1], 
                       [x_list[required_index]*x_list[required_index],x_list[required_index],1]])
    
    results = nmp.array([y_list[required_index-2],y_list[required_index-1],y_list[required_index]])
    ans = nmp.linalg.solve(coefficients, results)
    return (ans[0]*arg*arg+ans[1]*arg+ans[2])


def Lagrange_interpolation(arg,x_list,y_list):
        res = 0
        for i in range (0,len(y_list)):
            numerator=1; denominator=1
            for j in range(0,len(x_list)):
                if i!=j:
                    numerator*=(arg-x_list[j])
                    denominator*=(x_list[i]-x_list[j]) 
            res += (numerator/denominator)*y_list[i]
        return res

    
def newton_mixed(arg,x_list,y_list,ind0,ind1,ind2):
    first_pow_diff=first_div_diff(x_list[ind1],x_list[ind0],
                                  y_list[ind1],y_list[ind0])
    
    second_pow_diff=second_div_diff(x_list[ind2],x_list[ind1],x_list[ind0],
                                    y_list[ind2],y_list[ind1],y_list[ind0])
    
    return y_list[ind0]+(arg-x_list[ind0])* first_pow_diff +(arg-x_list[ind0])*(arg-x_list[ind1])*second_pow_diff
    

    
def newton_first(arg,x_list,y_list):
   
    required_index = get_necessary_index(arg,x_list)
    required_index -= 1
    res = y_list[required_index]
    itCounter = len(x_list)- 1 - required_index
    diffs = []
    h = round((x_list[1]-x_list[0]),10) 
    t = (arg-x_list[required_index])/h   
    
    for i in range(itCounter):
        diffs.append([])
    
    for i in range(required_index,len(x_list)-1):
        diffs[0].append(y_list[i+1]-y_list[i])
        
    for i in range(1,len(diffs)):
        for j in range(len(diffs[i-1])-1): 
            diffs[i].append(diffs[i-1][j+1] - diffs[i-1][j])
        
    for i in range(1,itCounter):
        acc = 1
        for j in range(i):
            acc*=(t-j)
            if (j!=0):
                acc/=j
        acc *= diffs[i-1][0]
        res += acc
                
    return res
    
def newton_second(arg,x_list,y_list):

    required_index = get_necessary_index(arg,x_list)
    res = y_list[required_index]
    itCounter = required_index
    diffs = []
    h = round((x_list[1]-x_list[0]),10) 
    t = (arg-x_list[required_index])/h 
    
    for i in range(itCounter):
        diffs.append([])
            
    for i in range(0,required_index):
        diffs[0].append(y_list[i+1]-y_list[i])
        
    for i in range(1,len(diffs)):
        for j in range(len(diffs[i-1])-1): 
            diffs[i].append(diffs[i-1][j+1] - diffs[i-1][j])
    
    for i in range(1,itCounter-1):
        acc = 1
        for j in range(i):
            acc*=(t+j)
            if (j!=0):
                acc/=j
        acc *= diffs[i-1][len(diffs[i-1])-i]
        res += acc
                
    return res

In [22]:
#main

print("Введите имя файла для считывания\n")

inpOK = True

try:
    while(True):
        testFilename = input()
        if os.path.exists(testFilename):
            break
        else:
            print("Такого файла не существует. Проверьте правильность ввода и попробуйте еще раз.")
        
    f3In = open(testFilename,'r')
    fileX_es = str(f3In.readline())
    fileY_es = str(f3In.readline())
    f3In.close()
    x_arr = fileX_es.split()
    y_arr = fileY_es.split()
    x_list=nmp.asarray(list(map(float,x_arr)))
    y_list=nmp.asarray(list(map(float,y_arr)))
    maxEl = nmp.amax(x_list)
    minEl = nmp.amin(x_list)
    if (len(x_list)!=len(y_list)):
        print("Несоответствующее количество Х и Y. Проверьте правильность данных и попробуйте еще раз.")
        inpOK = False
except ValueError as e:
    print("В файле найдены нечисловые данные. Проверьте правильность данных и попробуйте еще раз.")
    inpOK = False
    
if (inpOK):
    print("Успешное чтение данных.")
    print("Введите значение аргумента, для которого вычисляется приближенное значение функции")
    while(True):
        try:
            arg = float(input())
            if (arg>maxEl or arg<minEl):
                print("Число не входит в необходимый промежуток!")
            else:
                break
        except ValueError as e:
            print("Введите число!")
    print("Результат линейной интерполяции: " + str(linear_interpolation(arg,x_list,y_list)))
    print("Результат квадратичной интерполяции: " + str(polynomial_interpolation(arg,x_list,y_list)))
    print("Результат интерполяции с помощью многочлена Лагранжа: " + str(Lagrange_interpolation(arg,x_list,y_list)))
    try:
        print("Результат интерполяции с помощью неравноотстоящих узлов (0,1,2): " + str(newton_mixed(arg,x_list,y_list,0,1,2)))
        print("Результат интерполяции с помощью неравноотстоящих узлов (1,2,3): " + str(newton_mixed(arg,x_list,y_list,1,2,3)))
    except IndexError as r:
        print("Индекс вне массива - проверьте узлы") 
    if (check_for_newton(x_list)):
        if (formula_select(arg,x_list)==1):
            print("Аргумент в первой половине промежутка - берем первую формулу Ньютона: " + str(newton_first(arg,x_list,y_list)))
        else:
            print("Аргумент во второй половине промежутка - берем вторую формулу Ньютона: " + str(newton_second(arg,x_list,y_list)))
    else:
        print("Данная функция не состоит из равноотстоящих точек")

Введите имя файла для считывания

lab3input2.txt
Данные из файла получены
Введите значение аргумента, для которого вычисляется приближенное значение функции
1.557
Результат линейной интерполяции: 5.2529499999999985
Результат квадратичной интерполяции: 5.253268989999999
Результат интерполяции с помощью многочлена Лагранжа: 5.254844268587183
Результат интерполяции с помощью неравноотстоящих узлов (0,1,2): 5.284790799999985
Результат интерполяции с помощью неравноотстоящих узлов (1,2,3): 5.236685434999994
Аргумент во второй половине промежутка - берем вторую формулу Ньютона: 5.253232959103
